In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import plotly
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error,mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("C:/cygwin64/home/Pankaj.Pa.Kumar/GreyAtom Class/capstone/data/")

In [3]:
customer_data = pd.read_excel("Customer_Data.xlsx")
jtd = pd.read_csv("JTD.csv")

# Cleaning of JTD data

In [4]:
jtd.tail(10)

,Unnamed: 0,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value
5619474,5619474,962812,10,MISC_SRV07,NaN,MISC_OTHER CHARGES,P011,1.0,MIN,23387.34
5619475,5619475,962813,10,PAINT_SRV,ACCDBP0056,PAINTING FULL BODY - SOLID,P001,22000.0,MIN,26999.90
5619476,5619476,962813,20,ACCT_SRV,ACCDBP0068,DENTING ROOF,P001,1800.0,MIN,2399.99
5619477,5619477,962813,30,ACCT_SRV,ACCDBP0074,MINOR DENTING PER PANEL,P001,550.0,MIN,5549.98
5619478,5619478,962813,40,ACCT_SRV,ACCDBP0120,DENTING RADIATOR MOUNTING PANEL UPPER,P001,850.0,MIN,1050.00
5619479,5619479,962813,50,ACCT_SRV,ACCDBP0121,DENTING RADIATOR MOUNTING PANEL LOWER,P001,850.0,MIN,1050.00
5619480,5619480,962813,60,ACCT_SRV,ACCDBP0110,DENTING QUARTER PANEL RHS,P001,750.0,MIN,1250.00
5619481,5619481,962813,70,TOYOTA_SRV,TOINEL0029,CHARGING BATTERY & TESTING LOAD,P001,42.0,MIN,223.00
5619482,5619482,962813,80,ACCT_SRV,ACCDEL0093,REPAIR LHS / RHS TAIL LIGHT BOX,P001,450.0,MIN,450.00
5619483,5619483,962813,90,ACCT_SRV,ACCDBP0037,DENTING FRONT BUMPER,P001,1200.0,MIN,2149.99


In [5]:
jtd.isnull().sum()

Unnamed: 0                   0
DBM Order                    0
Order Item                   0
Material                 51629
Labor Value Number     3774033
Description              51629
Item Category                0
Order Quantity               0
Target quantity UoM      51629
Net value                    0
dtype: int64

In [6]:
aggregation_jtd_dbno={ 'Description' : {'Description' : lambda x: str(set(x)).strip('{}').replace("'","")},
                      'Item Category' : {'Item Category' : lambda x: str(set(x)).strip('{}').replace("'","")}, 
                      'Order Quantity' : {'Order Quantity' : 'sum'},
                      'Net value' : {'Net value' : 'sum'} }


In [7]:
jtd_grouped = jtd.groupby('DBM Order')['Description','Item Category','Order Quantity',
                                       'Net value'].agg(aggregation_jtd_dbno).reset_index()

In [8]:
jtd_grouped.head()

,DBM Order,Description,Item Category,Order Quantity,Net value
0,4,"VALVE 1-EXAUST, MACHINING CYLINDER HEAD (PETR...","P001, P002, P010, P011",731.6,16998.49
1,5,"REPLACE BRAKE PAD (BOTH SIDE), CLEAFUEL FILTER...","P001, P002",290.2,3779.80
2,6,"Paid Service-Minor, MISC LABOR 0.5 HOURS, REPL...",P001,109.0,1340.37
3,7,MISC LABOR 0.25 HOURS,P001,36.5,167.36
4,8,"DISTILLED WATER, ELF Synthetic Engine OIL, WAS...","P001, P002",69.0,2306.94


In [9]:
jtd_grouped.shape

(973473, 5)

In [10]:
jtd_grouped.isnull().sum()

DBM Order         0
Description       0
Item Category     0
Order Quantity    0
Net value         0
dtype: int64

# Cleaning of customer data

In [11]:
customer_data.head()

,Business Partner,Customer No.,Partner Type,Data Origin,Title,Marital Status,Occupation,Date of Birth,Death date
0,31,1,1.0,Z001,NaN,NaN,NaN,NaN,NaT
1,32,2,1.0,Z005,NaN,NaN,NaN,NaN,NaT
2,34,4,1.0,Z005,NaN,2.0,NaN,NaN,NaT
3,35,5,1.0,Z005,NaN,NaN,NaN,NaN,NaT
4,42,7,1.0,Z008,NaN,NaN,NaN,NaN,NaT


In [12]:
customer_data.isnull().sum()

Business Partner         0
Customer No.             0
Partner Type            78
Data Origin            136
Title                46307
Marital Status      539301
Occupation          540780
Date of Birth       541894
Death date          554675
dtype: int64

In [13]:
customer_data.shape

(555338, 9)

In [14]:
customer_data["Data Origin"].nunique()

22

In [15]:
customer_data["Partner Type"].value_counts()

1.0       554967
2.0           68
3.0           62
9001.0        57
9003.0        56
4.0           48
9002.0         2
Name: Partner Type, dtype: int64

In [16]:
customer_data.drop(["Marital Status", "Occupation", "Date of Birth", "Death date","Business Partner"],1,
                   inplace=True)

In [17]:
customer_data['Partner Type'] = customer_data['Partner Type'].fillna(customer_data['Partner Type'].mode()[0])

In [18]:
customer_data['Data Origin'] = customer_data['Data Origin'].fillna(customer_data['Data Origin'].mode()[0])

In [19]:
customer_data['Title'] = customer_data['Title'].fillna(customer_data['Title'].mode()[0])

In [20]:
customer_data.isnull().sum()

Customer No.    0
Partner Type    0
Data Origin     0
Title           0
dtype: int64

In [21]:
customer_data.head()

,Customer No.,Partner Type,Data Origin,Title
0,1,1.0,Z001,2.0
1,2,1.0,Z005,2.0
2,4,1.0,Z005,2.0
3,5,1.0,Z005,2.0
4,7,1.0,Z008,2.0


# Cleaning of final invoice

In [22]:
final_invoice = pd.read_csv("Final_invoice.csv",infer_datetime_format = True)

In [23]:
pd.set_option('display.max_columns', None)
final_invoice.head()

,Unnamed: 0,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,Claim No.,Cust Type,Customer No.,District,Expiry Date,Gate Pass Date,Gate Pass Time,IGST(12%),IGST(18%),IGST(28%),IGST(5%),Insurance Company,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,ODN No.,OSL Total,Order Type,Outstanding Amt,Parts Total,Pin code,Plant,Plant Name1,Policy no.,Print Status,Recovrbl Exp,Regn No,SGST/UGST(14%),SGST/UGST(2.5%),SGST/UGST(6%),SGST/UGST(9%),Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,0,0.0,0.0,MAJIWADA,0.0,0.0,0.0,0.0,Thane,NaN,NaN,Retail,67849,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,7.005200e+09,500.06,Paid Service,0.0,2348.75,400601,BC01,THANE,NaN,NO,0.0,KA19MA1291,0.0,0.0,0.0,0.0,NaN,0.0,RUPESH,4051.95,0.0,0.0,0.0,0.0,4051.95,BC01FS1
1,1,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,7.005200e+09,0.00,SMC Value Package,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,1001.29,0.0,0.0,0.0,0.0,1001.29,BC01SA2
2,2,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE[W],NaN,NaN,Retail,81055,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200004,11:12:57,173889,2015-01-02,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,7.005200e+09,0.00,Running Repairs,0.0,52.95,400607,BC01,THANE,NaN,NO,0.0,AP09AX0582,0.0,0.0,0.0,0.0,NaN,0.0,IMRAN,233.14,0.0,0.0,0.0,0.0,233.14,BC01SA2
3,3,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200005,11:40:44,174158,2015-01-03,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,7.005200e+09,0.00,SMC Redemption,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,0.00,0.0,0.0,0.0,0.0,0.00,BC01SA2
4,4,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE,NaN,NaN,Retail,18980,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200006,12:07:17,173860,2015-01-02,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,7.005200e+09,499.99,Paid Service,0.0,944.16,400603,BC01,THANE,NaN,NO,0.0,MH04CD9768,0.0,0.0,0.0,0.0,NaN,0.0,DAYANAND,2747.12,0.0,0.0,0.0,0.0,2747.12,BC01SA2


In [24]:
final_invoice["Job Card No"].nunique()

489329

In [25]:
final_invoice.isnull().sum()

Unnamed: 0                   0
Amt Rcvd From Custom    197286
Amt Rcvd From Ins Co    197286
Area / Locality          23890
CGST(14%)               197286
CGST(2.5%)              197286
CGST(6%)                197286
CGST(9%)                197286
CITY                         1
Cash /Cashless Type     464894
Claim No.               487510
Cust Type                    0
Customer No.                 0
District                     0
Expiry Date             470083
Gate Pass Date          451399
Gate Pass Time               0
IGST(12%)               197286
IGST(18%)               197286
IGST(28%)               197286
IGST(5%)                197286
Insurance Company       468723
Invoice Date                 0
Invoice No                   0
Invoice Time                 0
Job Card No                  0
JobCard Date                 0
JobCard Time                 0
KMs Reading                  0
Labour Total                 0
Make                         0
Misc Total                   0
Model   

In [26]:
columns_to_be_removed = final_invoice.isnull().sum()[final_invoice.isnull().sum().sort_values() > 30000].index

In [27]:
columns_to_be_removed

Index(['Amt Rcvd From Custom', 'Amt Rcvd From Ins Co', 'CGST(14%)',
       'CGST(2.5%)', 'CGST(6%)', 'CGST(9%)', 'Cash /Cashless Type',
       'Claim No.', 'Expiry Date', 'Gate Pass Date', 'IGST(12%)', 'IGST(18%)',
       'IGST(28%)', 'IGST(5%)', 'Insurance Company', 'ODN No.',
       'Outstanding Amt', 'Policy no.', 'SGST/UGST(14%)', 'SGST/UGST(2.5%)',
       'SGST/UGST(6%)', 'SGST/UGST(9%)', 'Service Advisor Name', 'TDS amount',
       'Technician Name', 'Total CGST', 'Total GST', 'Total IGST',
       'Total SGST/UGST', 'Total Value'],
      dtype='object')

In [28]:
final_invoice = final_invoice.drop(columns=columns_to_be_removed)

In [29]:
final_invoice.shape

(492314, 29)

In [30]:
final_invoice.head()

,Unnamed: 0,Area / Locality,CITY,Cust Type,Customer No.,District,Gate Pass Time,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Regn No,Total Amt Wtd Tax.,User ID
0,0,MAJIWADA,Thane,Retail,67849,Maharashtra,00:00:00,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,NO,0.0,KA19MA1291,4051.95,BC01FS1
1,1,THNAE,THNAE,Retail,84419,Maharashtra,00:00:00,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,0.00,SMC Value Package,0.00,400603,BC01,THANE,NO,0.0,MH43R3046,1001.29,BC01SA2
2,2,THANE,THANE[W],Retail,81055,Maharashtra,00:00:00,2015-01-03,7005200004,11:12:57,173889,2015-01-02,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,0.00,Running Repairs,52.95,400607,BC01,THANE,NO,0.0,AP09AX0582,233.14,BC01SA2
3,3,THNAE,THNAE,Retail,84419,Maharashtra,00:00:00,2015-01-03,7005200005,11:40:44,174158,2015-01-03,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,0.00,SMC Redemption,0.00,400603,BC01,THANE,NO,0.0,MH43R3046,0.00,BC01SA2
4,4,THANE,THANE,Retail,18980,Maharashtra,00:00:00,2015-01-03,7005200006,12:07:17,173860,2015-01-02,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,499.99,Paid Service,944.16,400603,BC01,THANE,NO,0.0,MH04CD9768,2747.12,BC01SA2


In [31]:
columns_irrelevant = ['Area / Locality', 'Print Status', 'Gate Pass Time', 'Recovrbl Exp']

In [32]:
final_invoice = final_invoice.drop(columns=columns_irrelevant)

In [33]:
final_invoice.dropna(subset=['Regn No'], inplace=True)

In [34]:
final_invoice.isnull().sum().sort_values()

Unnamed: 0               0
Regn No                  0
Plant Name1              0
Plant                    0
Pin code                 0
Parts Total              0
Order Type               0
OSL Total                0
Misc Total               0
Make                     0
Total Amt Wtd Tax.       0
Labour Total             0
JobCard Time             0
JobCard Date             0
Job Card No              0
Invoice Time             0
Invoice No               0
Invoice Date             0
District                 0
Customer No.             0
Cust Type                0
KMs Reading              0
User ID                  0
CITY                     1
Model                 1524
dtype: int64

In [35]:
final_invoice['CITY'] = final_invoice['CITY'].fillna(final_invoice['CITY'].mode()[0])
final_invoice['Model']  = final_invoice['Model'].fillna(final_invoice['Model'].mode()[0])

In [36]:
final_invoice.rename(columns= {"District" : "State"}, inplace=True)

In [37]:
final_invoice["Invoice month"] = pd.DatetimeIndex(final_invoice['Invoice Date']).month

In [38]:
final_invoice["Invoice year"] = pd.DatetimeIndex(final_invoice['Invoice Date']).year

In [39]:
final_invoice.drop(["Unnamed: 0"],1,inplace=True)

In [40]:
final_invoice.tail()

,CITY,Cust Type,Customer No.,State,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Regn No,Total Amt Wtd Tax.,User ID,Invoice month,Invoice year
492309,Chennai,Retail,203820,Tamil Nadu,2016-06-30,7231000017,14:56:58,400100,2016-06-30,09:15:12,72546,2495.90,MARUTI SUZUKI,374.97,ALTO,0.0,Paid Service,1587.28,600058,X405,X405 - NANGAN MOTORS PVT. LTD.,TN38Y9343,4458.15,X405SA1,6,2016
492310,ADAMBAKKAM,Retail,170411,Tamil Nadu,2016-06-30,7231000018,15:06:52,400092,2016-06-30,08:42:15,18968,2443.47,NISSAN,0.00,MICRA,0.0,Paid Service,4388.61,600088,X405,X405 - NANGAN MOTORS PVT. LTD.,TN22CK0396,6832.08,X405SA1,6,2016
492311,CHENNAI,Retail,48811,Tamil Nadu,2016-06-30,7231000019,15:43:56,400082,2016-06-30,08:27:51,177433,2598.92,MAHINDRA & MAHINDRA,0.00,SCORPIO CRDE,0.0,Paid Service,6588.79,600044,X405,X405 - NANGAN MOTORS PVT. LTD.,TN07BA0211,9187.71,X405SA1,6,2016
492312,"Vijaya Nagar, 1 main road,Velacherr",Retail,204067,Tamil Nadu,2016-06-30,7231000020,15:58:59,400545,2016-06-30,12:53:48,22932,11000.00,HONDA,100.00,AMAZE,0.0,Accidental,0.00,600042,X405,X405 - NANGAN MOTORS PVT. LTD.,TN07BV9290,11100.00,X405SA1,6,2016
492313,saidapet,Retail,204288,Tamil Nadu,2016-06-30,7231000021,17:09:19,400891,2016-06-30,16:54:06,149197,747.78,MARUTI SUZUKI,0.00,WAGONR,0.0,Running Repairs,0.00,600016,X405,X405 - NANGAN MOTORS PVT. LTD.,TN10T3317,747.78,X405WM1,6,2016


# Cleaning of plant master

In [41]:
plant_master = pd.read_excel("PlantMaster.xlsx")

In [42]:
plant_master.head()

,Plant,Name 1,Valuation Area,Customer no. - plant,Vendor number plant,Factory calendar,Name 2,House number and street,PO Box,Postal Code,City,Sales organization,State
0,BC07,NERKUNDRAM,BC07,PLBC07,PLBC07,IN,NaN,"No.117 Poonamallee High Road,",600107.0,600107,Chennai,MFCS,Tamil Nadu
1,BC30,SAS NAGAR - MOHALI,BC30,PLBC30,PLBC30,IN,NaN,"C-118, Phase 07,",160055.0,160055,Mohali,MFCS,Punjab
2,X001,X001 - AUTO PLANET WHEELS,X001,PLX001,NaN,IN,Thane,"Unit No.3, Kothari Compound,",400610.0,400610,Thane,MFCA,Maharashtra
3,X002,X002 - BHANDARI INDUSTRIES PVT,X002,PLX002,NaN,IN,Ghaziabad,"18/1, C Site 4, Sahibabad",201010.0,201010,Ghaziabad,MFCA,Uttar Pradesh
4,X003,X003 - Om Sri Satya Sai Automo,X003,PLX003,NaN,IN,Bengaluru,"#75, Sy.No. 83/2, Panathur Mai",560087.0,560087,Bengaluru,MFCB,Karnataka


In [43]:
plant_master.isnull().sum()

Plant                        0
Name 1                       0
Valuation Area               0
Customer no. - plant         0
Vendor number plant        436
Factory calendar             0
Name 2                       2
House number and street      2
PO Box                       1
Postal Code                  0
City                         0
Sales organization           5
State                        0
dtype: int64

In [44]:
plant_master.shape

(438, 13)

In [45]:
plant = plant_master.drop(columns=['Vendor number plant', 'Valuation Area',
                                      'Factory calendar', 'Name 2', 'House number and street',
                                      'PO Box', 'Sales organization', 'Customer no. - plant'])

In [46]:
plant.shape

(438, 5)

In [47]:
plant.isnull().sum()

Plant          0
Name 1         0
Postal Code    0
City           0
State          0
dtype: int64

In [48]:
plant.head()

,Plant,Name 1,Postal Code,City,State
0,BC07,NERKUNDRAM,600107,Chennai,Tamil Nadu
1,BC30,SAS NAGAR - MOHALI,160055,Mohali,Punjab
2,X001,X001 - AUTO PLANET WHEELS,400610,Thane,Maharashtra
3,X002,X002 - BHANDARI INDUSTRIES PVT,201010,Ghaziabad,Uttar Pradesh
4,X003,X003 - Om Sri Satya Sai Automo,560087,Bengaluru,Karnataka


# Merging data

In [49]:
final_invoice['Customer No.'] = final_invoice['Customer No.'].str.lstrip('0')

In [50]:
customer_data['Customer No.'] = customer_data['Customer No.'].astype(str)

In [51]:
invoice_customer = pd.merge(final_invoice, customer_data, on='Customer No.')

In [52]:
invoice_customer.shape, len(final_invoice['Customer No.'].unique()), len(customer_data['Customer No.'].unique()), len(invoice_customer['Customer No.'].unique())

((492296, 29), 253480, 555338, 253478)

In [53]:
invoice_customer.isnull().sum()

CITY                  0
Cust Type             0
Customer No.          0
State                 0
Invoice Date          0
Invoice No            0
Invoice Time          0
Job Card No           0
JobCard Date          0
JobCard Time          0
KMs Reading           0
Labour Total          0
Make                  0
Misc Total            0
Model                 0
OSL Total             0
Order Type            0
Parts Total           0
Pin code              0
Plant                 0
Plant Name1           0
Regn No               0
Total Amt Wtd Tax.    0
User ID               0
Invoice month         0
Invoice year          0
Partner Type          0
Data Origin           0
Title                 0
dtype: int64

In [54]:
invoice_customer.head()

,CITY,Cust Type,Customer No.,State,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Regn No,Total Amt Wtd Tax.,User ID,Invoice month,Invoice year,Partner Type,Data Origin,Title
0,Thane,Retail,67849,Maharashtra,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.0,SPARK,500.06,Paid Service,2348.75,400601,BC01,THANE,KA19MA1291,4051.95,BC01FS1,1,2015,1.0,Z007,2.0
1,Thane,Retail,67849,Maharashtra,2014-08-14,7000137359,17:54:30,136482,2014-08-08,15:26:53,45575,989.98,GENERAL MOTORS,0.0,SPARK,6249.87,Accidental,4230.25,400601,BC01,THANE,KA19MA1291,11470.10,BC01FS1,8,2014,1.0,Z007,2.0
2,Thane,Retail,67849,Maharashtra,2014-08-16,7000137489,10:52:30,135895,2014-08-06,12:49:48,45575,6473.05,GENERAL MOTORS,0.0,SPARK,7325.09,Paid Service,14546.14,400601,BC01,THANE,KA19MA1291,28344.28,BC01FS1,8,2014,1.0,Z007,2.0
3,Thane,Retail,67849,Maharashtra,2014-10-09,7000151113,17:35:30,150383,2014-10-06,11:40:52,47556,0.00,GENERAL MOTORS,0.0,SPARK,0.00,Running Repairs,200.00,400601,BC01,THANE,KA19MA1291,200.00,BC01SA1,10,2014,1.0,Z007,2.0
4,Thane,Retail,67849,Maharashtra,2014-12-31,7000175211,18:41:00,168210,2014-12-13,12:14:37,49317,1760.02,GENERAL MOTORS,500.0,SPARK,7850.09,Accidental,20348.81,400601,BC01,THANE,KA19MA1291,30458.92,BC01FS1,12,2014,1.0,Z007,2.0


In [55]:
jtd_grouped.head()

,DBM Order,Description,Item Category,Order Quantity,Net value
0,4,"VALVE 1-EXAUST, MACHINING CYLINDER HEAD (PETR...","P001, P002, P010, P011",731.6,16998.49
1,5,"REPLACE BRAKE PAD (BOTH SIDE), CLEAFUEL FILTER...","P001, P002",290.2,3779.80
2,6,"Paid Service-Minor, MISC LABOR 0.5 HOURS, REPL...",P001,109.0,1340.37
3,7,MISC LABOR 0.25 HOURS,P001,36.5,167.36
4,8,"DISTILLED WATER, ELF Synthetic Engine OIL, WAS...","P001, P002",69.0,2306.94


In [56]:
plant.head()

,Plant,Name 1,Postal Code,City,State
0,BC07,NERKUNDRAM,600107,Chennai,Tamil Nadu
1,BC30,SAS NAGAR - MOHALI,160055,Mohali,Punjab
2,X001,X001 - AUTO PLANET WHEELS,400610,Thane,Maharashtra
3,X002,X002 - BHANDARI INDUSTRIES PVT,201010,Ghaziabad,Uttar Pradesh
4,X003,X003 - Om Sri Satya Sai Automo,560087,Bengaluru,Karnataka


In [57]:
plant.shape

(438, 5)

In [58]:
plant.rename(columns= {"State" : "State_Plant"}, inplace=True)

In [59]:
invoice_customer_plant = pd.merge(invoice_customer,plant, on='Plant')

In [60]:
df_plant_check = invoice_customer_plant.copy()

In [61]:
invoice_customer_plant.head(5)

,CITY,Cust Type,Customer No.,State,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Regn No,Total Amt Wtd Tax.,User ID,Invoice month,Invoice year,Partner Type,Data Origin,Title,Name 1,Postal Code,City,State_Plant
0,THANE,Retail,80479,Maharashtra,2015-06-19,7151201169,18:10:34,227460,2015-06-19,13:56:32,65714,1802.05,FORD,0.0,FIESTA,0.0,Paid Service,399.73,600406,X006,X006 - A.M.MOTORS,MH03AB0789,2201.78,X006SA1,6,2015,1.0,Z010,2.0,X006 - A.M.MOTORS,401104,Thane,Maharashtra
1,Thane,Retail,E15810,Maharashtra,2016-07-20,7151205340,10:26:00,413455,2016-07-18,15:49:34,53903,2874.80,MARUTI SUZUKI,0.0,SWIFT,0.0,Running Repairs,3151.06,421301,X006,X006 - A.M.MOTORS,MH04CT295,6025.86,X006WM1,7,2016,1.0,Z001,2.0,X006 - A.M.MOTORS,401104,Thane,Maharashtra
2,Thane,Retail,E15810,Maharashtra,2016-09-04,7151205916,11:12:47,446448,2016-08-30,10:10:59,54231,2799.96,MARUTI SUZUKI,0.0,SWIFT,0.0,Running Repairs,664.04,421301,X006,X006 - A.M.MOTORS,MH04CT295,3464.00,X006WM1,9,2016,1.0,Z001,2.0,X006 - A.M.MOTORS,401104,Thane,Maharashtra
3,NAVI MUMBAI,Retail,73857,Maharashtra,2016-10-28,7151206592,13:17:54,487708,2016-10-18,17:47:35,55118,554.95,HYUNDAI,0.0,VERNA,0.0,Running Repairs,1510.80,400096,X006,X006 - A.M.MOTORS,MH43AN1186,2065.75,X006SA1,10,2016,1.0,Z001,2.0,X006 - A.M.MOTORS,401104,Thane,Maharashtra
4,Thane,Retail,E25519,Maharashtra,2015-05-28,7151201014,18:20:37,220109,2015-05-28,17:57:40,182687,75.02,MAHINDRA & MAHINDRA,0.0,LOGAN D,0.0,Running Repairs,578.43,400607,X006,X006 - A.M.MOTORS,MH04DN6210,653.45,X006SA1,5,2015,1.0,Z001,2.0,X006 - A.M.MOTORS,401104,Thane,Maharashtra


In [62]:
invoice_customer_plant["Name 1"].value_counts().head()

NERKUNDRAM                     11815
SAS NAGAR - MOHALI              7403
X006 - A.M.MOTORS               6505
X025 - SHIVALIK AUTOMOBILES     5497
X013 - SAI AUTO WORKS           4971
Name: Name 1, dtype: int64

In [63]:
invoice_customer_plant.drop(["Postal Code","Plant","City","State_Plant"],1, inplace = True)

In [64]:
invoice_customer_plant.head()

,CITY,Cust Type,Customer No.,State,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant Name1,Regn No,Total Amt Wtd Tax.,User ID,Invoice month,Invoice year,Partner Type,Data Origin,Title,Name 1
0,THANE,Retail,80479,Maharashtra,2015-06-19,7151201169,18:10:34,227460,2015-06-19,13:56:32,65714,1802.05,FORD,0.0,FIESTA,0.0,Paid Service,399.73,600406,X006 - A.M.MOTORS,MH03AB0789,2201.78,X006SA1,6,2015,1.0,Z010,2.0,X006 - A.M.MOTORS
1,Thane,Retail,E15810,Maharashtra,2016-07-20,7151205340,10:26:00,413455,2016-07-18,15:49:34,53903,2874.80,MARUTI SUZUKI,0.0,SWIFT,0.0,Running Repairs,3151.06,421301,X006 - A.M.MOTORS,MH04CT295,6025.86,X006WM1,7,2016,1.0,Z001,2.0,X006 - A.M.MOTORS
2,Thane,Retail,E15810,Maharashtra,2016-09-04,7151205916,11:12:47,446448,2016-08-30,10:10:59,54231,2799.96,MARUTI SUZUKI,0.0,SWIFT,0.0,Running Repairs,664.04,421301,X006 - A.M.MOTORS,MH04CT295,3464.00,X006WM1,9,2016,1.0,Z001,2.0,X006 - A.M.MOTORS
3,NAVI MUMBAI,Retail,73857,Maharashtra,2016-10-28,7151206592,13:17:54,487708,2016-10-18,17:47:35,55118,554.95,HYUNDAI,0.0,VERNA,0.0,Running Repairs,1510.80,400096,X006 - A.M.MOTORS,MH43AN1186,2065.75,X006SA1,10,2016,1.0,Z001,2.0,X006 - A.M.MOTORS
4,Thane,Retail,E25519,Maharashtra,2015-05-28,7151201014,18:20:37,220109,2015-05-28,17:57:40,182687,75.02,MAHINDRA & MAHINDRA,0.0,LOGAN D,0.0,Running Repairs,578.43,400607,X006 - A.M.MOTORS,MH04DN6210,653.45,X006SA1,5,2015,1.0,Z001,2.0,X006 - A.M.MOTORS


In [65]:
invoice_customer_plant["KMs Reading"] = invoice_customer_plant["KMs Reading"].astype(int)
invoice_customer_plant.loc[invoice_customer_plant["KMs Reading"] <= 100000, "KMs Reading"] = 0

invoice_customer_plant.loc[(invoice_customer_plant["KMs Reading"] > 100000) 
                           & (invoice_customer_plant["KMs Reading"] <= 200000), "KMs Reading"] = 1

invoice_customer_plant.loc[(invoice_customer_plant["KMs Reading"] > 200000) 
                           & (invoice_customer_plant["KMs Reading"] <= 300000), "KMs Reading"] = 2
invoice_customer_plant.loc[(invoice_customer_plant["KMs Reading"] > 300000) 
                           & (invoice_customer_plant["KMs Reading"] <= 400000), "KMs Reading"] = 3

invoice_customer_plant.loc[(invoice_customer_plant["KMs Reading"] > 400000) 
                           & (invoice_customer_plant["KMs Reading"] <= 500000), "KMs Reading"] = 4

invoice_customer_plant.loc[(invoice_customer_plant["KMs Reading"] > 500000) 
                           & (invoice_customer_plant["KMs Reading"] <= 600000), "KMs Reading"] = 5

invoice_customer_plant.loc[(invoice_customer_plant["KMs Reading"] > 600000) 
                           & (invoice_customer_plant["KMs Reading"] <= 700000), "KMs Reading"] = 6

invoice_customer_plant.loc[invoice_customer_plant["KMs Reading"] > 700000, "KMs Reading"] = 7


In [66]:
invoice_customer_plant["KMs Reading"].value_counts()

0    191538
1     52137
2      7720
3      1680
7      1508
4       541
5       343
6       287
Name: KMs Reading, dtype: int64

In [67]:
invoice_customer_plant.head()

,CITY,Cust Type,Customer No.,State,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant Name1,Regn No,Total Amt Wtd Tax.,User ID,Invoice month,Invoice year,Partner Type,Data Origin,Title,Name 1
0,THANE,Retail,80479,Maharashtra,2015-06-19,7151201169,18:10:34,227460,2015-06-19,13:56:32,0,1802.05,FORD,0.0,FIESTA,0.0,Paid Service,399.73,600406,X006 - A.M.MOTORS,MH03AB0789,2201.78,X006SA1,6,2015,1.0,Z010,2.0,X006 - A.M.MOTORS
1,Thane,Retail,E15810,Maharashtra,2016-07-20,7151205340,10:26:00,413455,2016-07-18,15:49:34,0,2874.80,MARUTI SUZUKI,0.0,SWIFT,0.0,Running Repairs,3151.06,421301,X006 - A.M.MOTORS,MH04CT295,6025.86,X006WM1,7,2016,1.0,Z001,2.0,X006 - A.M.MOTORS
2,Thane,Retail,E15810,Maharashtra,2016-09-04,7151205916,11:12:47,446448,2016-08-30,10:10:59,0,2799.96,MARUTI SUZUKI,0.0,SWIFT,0.0,Running Repairs,664.04,421301,X006 - A.M.MOTORS,MH04CT295,3464.00,X006WM1,9,2016,1.0,Z001,2.0,X006 - A.M.MOTORS
3,NAVI MUMBAI,Retail,73857,Maharashtra,2016-10-28,7151206592,13:17:54,487708,2016-10-18,17:47:35,0,554.95,HYUNDAI,0.0,VERNA,0.0,Running Repairs,1510.80,400096,X006 - A.M.MOTORS,MH43AN1186,2065.75,X006SA1,10,2016,1.0,Z001,2.0,X006 - A.M.MOTORS
4,Thane,Retail,E25519,Maharashtra,2015-05-28,7151201014,18:20:37,220109,2015-05-28,17:57:40,1,75.02,MAHINDRA & MAHINDRA,0.0,LOGAN D,0.0,Running Repairs,578.43,400607,X006 - A.M.MOTORS,MH04DN6210,653.45,X006SA1,5,2015,1.0,Z001,2.0,X006 - A.M.MOTORS


In [68]:
invoice_customer_plant.shape, invoice_customer_plant.columns

((255754, 29),
 Index(['CITY', 'Cust Type', 'Customer No.', 'State', 'Invoice Date',
        'Invoice No', 'Invoice Time', 'Job Card No', 'JobCard Date',
        'JobCard Time', 'KMs Reading', 'Labour Total', 'Make', 'Misc Total',
        'Model', 'OSL Total', 'Order Type', 'Parts Total', 'Pin code',
        'Plant Name1', 'Regn No', 'Total Amt Wtd Tax.', 'User ID',
        'Invoice month', 'Invoice year', 'Partner Type', 'Data Origin', 'Title',
        'Name 1'],
       dtype='object'))

In [69]:
invoice_customer_plant['Invoice Date'] = pd.to_datetime(invoice_customer_plant['Invoice Date'])

In [70]:
invoice_customer_plant['Invoice Date Time'] = invoice_customer_plant['Invoice Date'].astype(str).str.cat(invoice_customer_plant[['Invoice Time']].astype(str),sep=' ')

In [71]:
invoice_customer_plant['JobCard Date'] =  pd.to_datetime(invoice_customer_plant['JobCard Date'])

In [72]:
invoice_customer_plant['JobCard Date Time'] = invoice_customer_plant['JobCard Date'].astype(str).str.cat(invoice_customer_plant[['JobCard Time']].astype(str), sep=' ')

In [73]:
invoice_customer_plant.loc[:, ['JobCard Date Time', 'JobCard Date','JobCard Time']].head()

,JobCard Date Time,JobCard Date,JobCard Time
0,2015-06-19 13:56:32,2015-06-19,13:56:32
1,2016-07-18 15:49:34,2016-07-18,15:49:34
2,2016-08-30 10:10:59,2016-08-30,10:10:59
3,2016-10-18 17:47:35,2016-10-18,17:47:35
4,2015-05-28 17:57:40,2015-05-28,17:57:40


In [74]:
invoice_customer_plant.loc[:, ['Invoice Date Time', 'Invoice Date','Invoice Time']].head()

,Invoice Date Time,Invoice Date,Invoice Time
0,2015-06-19 18:10:34,2015-06-19,18:10:34
1,2016-07-20 10:26:00,2016-07-20,10:26:00
2,2016-09-04 11:12:47,2016-09-04,11:12:47
3,2016-10-28 13:17:54,2016-10-28,13:17:54
4,2015-05-28 18:20:37,2015-05-28,18:20:37


In [75]:
invoice_customer_plant['Invoice Date Time'] = pd.to_datetime(invoice_customer_plant['Invoice Date Time'])

In [76]:
invoice_customer_plant['JobCard Date Time'] = pd.to_datetime(invoice_customer_plant['JobCard Date Time'])

In [77]:
invoice_customer_plant['ServiceTime'] = invoice_customer_plant['Invoice Date Time']-invoice_customer_plant['JobCard Date Time']

In [78]:
invoice_customer_plant.loc[:, ['ServiceTime']].head()

,ServiceTime
0,0 days 04:14:02
1,1 days 18:36:26
2,5 days 01:01:48
3,9 days 19:30:19
4,0 days 00:22:57


In [79]:
invoice_customer_plant.shape

(255754, 32)

In [80]:
invoice_customer_plant = invoice_customer_plant.loc[invoice_customer_plant['ServiceTime']>pd.Timedelta(hours=0),:]

In [81]:
invoice_customer_plant.shape

(255753, 32)

In [82]:
invoice_customer_plant["Order Type"].value_counts()

Running Repairs      145444
Paid Service          72424
Accidental            21241
Mechanical            12910
SMC Redemption         2028
SMC Value Package      1431
Repeat Order            258
WBW Order                17
Name: Order Type, dtype: int64

In [83]:
invoice_customer_plant['ServiceTime_category']=None
invoice_customer_plant.loc[invoice_customer_plant['ServiceTime']<pd.Timedelta(hours=13),'ServiceTime_category']='super_fast'
invoice_customer_plant.loc[(invoice_customer_plant['ServiceTime']>=pd.Timedelta(hours=13))&(invoice_customer_plant['ServiceTime']<pd.Timedelta(hours=36)),'ServiceTime_category']='fast'
invoice_customer_plant.loc[(invoice_customer_plant['ServiceTime']>=pd.Timedelta(hours=36))&(invoice_customer_plant['ServiceTime']<pd.Timedelta(hours=61)),'ServiceTime_category']='mid'
invoice_customer_plant.loc[(invoice_customer_plant['ServiceTime']>=pd.Timedelta(hours=61))&(invoice_customer_plant['ServiceTime']<pd.Timedelta(hours=84)),'ServiceTime_category']='slow'
invoice_customer_plant.loc[invoice_customer_plant['ServiceTime']>=pd.Timedelta(hours=84),'ServiceTime_category']='super_slow'

In [84]:
# invoice_customer_plant.drop(columns=
#                             ['Invoice Time', 'Invoice Date', 'Invoice No', 'JobCard Date', 'JobCard Time','Model','Pin code','Plant Name1','User ID','Name 1','CITY','Invoice Date Time','JobCard Date Time','ServiceTime'],
#                             inplace=True)
invoice_customer_plant.drop(columns=
                            ['Invoice Time', 'Invoice Date', 'JobCard Date', 'JobCard Time','Model','Pin code','Plant Name1','User ID','Name 1','CITY','Invoice Date Time','JobCard Date Time','ServiceTime'],
                            inplace=True)

# Merging jtd and invoice data

In [85]:
len(invoice_customer_plant['Job Card No'].unique()), len(jtd_grouped['DBM Order'].unique())

(253199, 973473)

In [86]:
invoice_customer_plant_jtd = pd.merge(invoice_customer_plant, jtd_grouped, 
                                               left_on='Job Card No', right_on='DBM Order').drop('DBM Order', axis=1)

In [87]:
invoice_customer_plant_jtd.head()

,Cust Type,Customer No.,State,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,OSL Total,Order Type,Parts Total,Regn No,Total Amt Wtd Tax.,Invoice month,Invoice year,Partner Type,Data Origin,Title,ServiceTime_category,Description,Item Category,Order Quantity,Net value
0,Retail,80479,Maharashtra,7151201169,227460,0,1802.05,FORD,0.0,0.0,Paid Service,399.73,MH03AB0789,2201.78,6,2015,1.0,Z010,2.0,super_fast,"INSULATION TAPE, BULB H1,12V,80W,P145S, DISTIL...","P001, P002",45.5,2201.78
1,Retail,E15810,Maharashtra,7151205340,413455,0,2874.80,MARUTI SUZUKI,0.0,0.0,Running Repairs,3151.06,MH04CT295,6025.86,7,2016,1.0,Z001,2.0,mid,"CLIP, MISC LABOR 0.25 HOURS, REPLACE BRAKE PAD...","P001, P002",202.5,6025.86
2,Retail,E15810,Maharashtra,7151205916,446448,0,2799.96,MARUTI SUZUKI,0.0,0.0,Running Repairs,664.04,MH04CT295,3464.00,9,2016,1.0,Z001,2.0,super_slow,"MISC LABOR 0.25 HOURS, CARBURETTOR SPRAY, ANAB...","P001, P002",189.0,3464.00
3,Retail,73857,Maharashtra,7151206592,487708,0,554.95,HYUNDAI,0.0,0.0,Running Repairs,1510.80,MH43AN1186,2065.75,10,2016,1.0,Z001,2.0,super_slow,"Paid Service-Minor, MFC PROLEUM C14/SL 15W40, ...","P001, P002",7.0,2065.75
4,Retail,E25519,Maharashtra,7151201014,220109,1,75.02,MAHINDRA & MAHINDRA,0.0,0.0,Running Repairs,578.43,MH04DN6210,653.45,5,2015,1.0,Z001,2.0,super_fast,"NOZZLE-LH WS WASHER, REMOVE & REFIT WIPER BLAD...","P001, P002",13.0,653.45


In [88]:
invoice_customer_plant_jtd.isnull().sum()

Cust Type               0
Customer No.            0
State                   0
Invoice No              0
Job Card No             0
KMs Reading             0
Labour Total            0
Make                    0
Misc Total              0
OSL Total               0
Order Type              0
Parts Total             0
Regn No                 0
Total Amt Wtd Tax.      0
Invoice month           0
Invoice year            0
Partner Type            0
Data Origin             0
Title                   0
ServiceTime_category    0
Description             0
Item Category           0
Order Quantity          0
Net value               0
dtype: int64

In [89]:
invoice_customer_plant_jtd.shape

(255714, 24)

#  LTV
To calculate customer lifetime value you need to calculate average purchase value, and 
then multiply that number by the average purchase frequency rate to determine customer value. 
Then, once you calculate average customer lifespan, you can multiply that by customer value to 
determine customer lifetime value.

LTV measures the projected revenue from a customer over the lifetime of their relationship with your business. Knowing the value of the repeat business helps you determine how much you should invest in customer retention and acquisition.

LTV equals Lifetime Customer Revenue minus Lifetime Customer Costs. Assuming cost as zero in this project.
High LTV customers can be rewarded (and retained) in a number of ways, such as:

1) Offering special discounts on multiple purchases
2) Creating a loyalty program (punch or swipe cards are popular)
3) Offering rewards for new customer referrals
4) Providing special customer service
5) Offering preferential credit terms

### According to Consumer Reports, the average lifespan of today's cars is about eight years. 

Considering LTV for each customer over a period of 2015 (1 year)

In [162]:
invoice_customer_plant_jtd.head()

,Cust Type,Customer No.,State,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,OSL Total,Order Type,Parts Total,Regn No,Total Amt Wtd Tax.,Invoice month,Invoice year,Partner Type,Data Origin,Title,ServiceTime_category,Description,Item Category,Order Quantity,Net value
0,Retail,80479,Maharashtra,7151201169,227460,0,1802.05,FORD,0.0,0.0,Paid Service,399.73,MH03AB0789,2201.78,6,2015,1.0,Z010,2.0,super_fast,"Paid Service-Minor, REPLACE HEADLIGHT HOLDER (...","P002, P001",45.5,2201.78
1,Retail,E15810,Maharashtra,7151205340,413455,0,2874.80,MARUTI SUZUKI,0.0,0.0,Running Repairs,3151.06,MH04CT295,6025.86,7,2016,1.0,Z001,2.0,mid,"NEW Engine oil flush, HANDLE ASSY,BACK DOO, AN...","P002, P001",202.5,6025.86
2,Retail,E15810,Maharashtra,7151205916,446448,0,2799.96,MARUTI SUZUKI,0.0,0.0,Running Repairs,664.04,MH04CT295,3464.00,9,2016,1.0,Z001,2.0,super_slow,"CARBURETTOR SPRAY, MISC LABOR 0.5 HOURS, REPLA...","P002, P001",189.0,3464.00
3,Retail,73857,Maharashtra,7151206592,487708,0,554.95,HYUNDAI,0.0,0.0,Running Repairs,1510.80,MH43AN1186,2065.75,10,2016,1.0,Z001,2.0,super_slow,"OIL FILTER VENTO, Paid Service-Minor, MFC PROL...","P002, P001",7.0,2065.75
4,Retail,E25519,Maharashtra,7151201014,220109,1,75.02,MAHINDRA & MAHINDRA,0.0,0.0,Running Repairs,578.43,MH04DN6210,653.45,5,2015,1.0,Z001,2.0,super_fast,"NOZZLE-LH WS WASHER, REMOVE & REFIT WIPER BLAD...","P002, P001",13.0,653.45


In [90]:
invoice_customer_plant_jtd.columns

Index(['Cust Type', 'Customer No.', 'State', 'Invoice No', 'Job Card No',
       'KMs Reading', 'Labour Total', 'Make', 'Misc Total', 'OSL Total',
       'Order Type', 'Parts Total', 'Regn No', 'Total Amt Wtd Tax.',
       'Invoice month', 'Partner Type', 'Data Origin', 'Title',
       'ServiceTime_category', 'Description', 'Item Category',
       'Order Quantity', 'Net value'],
      dtype='object')

In [90]:
customer_15=invoice_customer_plant_jtd[invoice_customer_plant_jtd["Invoice year"]==2015]

In [91]:
customer_freq=customer_15.groupby(["Customer No.", "Make"]) \
            .agg({"Invoice No":'nunique', "Total Amt Wtd Tax.":'mean'})
customer_freq=customer_freq.reset_index()
customer_freq.tail(7)

,Customer No.,Make,Invoice No,Total Amt Wtd Tax.
41253,MFCC018,TATA MOTORS,2,17912.005000
41254,MFCC036,MARUTI SUZUKI,1,2665.520000
41255,MFCCM01,MAHINDRA & MAHINDRA,2,12018.565000
41256,MFCCM03,MAHINDRA & MAHINDRA,81,6387.137654
41257,MFCCM14,HONDA,1,192478.010000
41258,MFCCM14,MAHINDRA & MAHINDRA,5,33203.484000
41259,MFCCM19,MAHINDRA & MAHINDRA,5,23233.206000


In [92]:
ltv_make=customer_freq.copy()
ltv_make["LTV"] = ltv_make["Invoice No"] * ltv_make["Total Amt Wtd Tax."]
ltv_make.tail(7)

,Customer No.,Make,Invoice No,Total Amt Wtd Tax.,LTV
41253,MFCC018,TATA MOTORS,2,17912.005000,35824.01
41254,MFCC036,MARUTI SUZUKI,1,2665.520000,2665.52
41255,MFCCM01,MAHINDRA & MAHINDRA,2,12018.565000,24037.13
41256,MFCCM03,MAHINDRA & MAHINDRA,81,6387.137654,517358.15
41257,MFCCM14,HONDA,1,192478.010000,192478.01
41258,MFCCM14,MAHINDRA & MAHINDRA,5,33203.484000,166017.42
41259,MFCCM19,MAHINDRA & MAHINDRA,5,23233.206000,116166.03


## TOP 20 special customer with highest LTV

In [93]:
ltv_make.sort_values('LTV', ascending=False).head(20)

,Customer No.,Make,Invoice No,Total Amt Wtd Tax.,LTV
41199,E2003,MAHINDRA & MAHINDRA,50,18883.733800,944186.69
36766,90834,SKODA,1,565487.200000,565487.20
41256,MFCCM03,MAHINDRA & MAHINDRA,81,6387.137654,517358.15
31943,69319,GENERAL MOTORS,7,66702.868571,466920.08
4125,107626,MAHINDRA & MAHINDRA,4,112174.367500,448697.47
13070,121624,MAHINDRA & MAHINDRA,1,419840.660000,419840.66
11141,118717,BMW,1,375616.720000,375616.72
31398,51811,GENERAL MOTORS,5,71308.050000,356540.25
39421,96440,GENERAL MOTORS,5,69760.652000,348803.26
3656,106757,MAHINDRA & MAHINDRA,1,335994.480000,335994.48


In [94]:
invoice_customer_plant_jtd.shape, ltv_make.shape

((255714, 24), (41260, 5))

In [95]:
new_df = pd.merge(invoice_customer_plant_jtd, ltv_make,  how='left', left_on=["Customer No.", "Make"], \
              right_on = ["Customer No.", "Make"])
new_df.shape, new_df.columns

((255714, 27),
 Index(['Cust Type', 'Customer No.', 'State', 'Invoice No_x', 'Job Card No',
        'KMs Reading', 'Labour Total', 'Make', 'Misc Total', 'OSL Total',
        'Order Type', 'Parts Total', 'Regn No', 'Total Amt Wtd Tax._x',
        'Invoice month', 'Invoice year', 'Partner Type', 'Data Origin', 'Title',
        'ServiceTime_category', 'Description', 'Item Category',
        'Order Quantity', 'Net value', 'Invoice No_y', 'Total Amt Wtd Tax._y',
        'LTV'],
       dtype='object'))

In [96]:
new_df = new_df[new_df['LTV'] > 0]

In [97]:
new_df2=new_df[new_df["Invoice year"]==2015]

In [98]:
# df = invoice_customer_plant_jtd.drop(["Job Card No","Labour Total","Misc Total","OSL Total",
#                                       "Parts Total","Regn No","Data Origin","Description","Net value","clusters"],1)
df = new_df2.drop(["Job Card No","Labour Total","Misc Total","OSL Total", "Invoice No_x", "Customer No.", \
                                      "Total Amt Wtd Tax._x", "Total Amt Wtd Tax._y", "Invoice No_y", "Invoice year", \
                                      "Parts Total","Regn No","Data Origin","Description","Net value"],1)

In [99]:
df.head()

,Cust Type,State,KMs Reading,Make,Order Type,Invoice month,Partner Type,Title,ServiceTime_category,Item Category,Order Quantity,LTV
0,Retail,Maharashtra,0,FORD,Paid Service,6,1.0,2.0,super_fast,"P001, P002",45.5,2201.78
4,Retail,Maharashtra,1,MAHINDRA & MAHINDRA,Running Repairs,5,1.0,2.0,super_fast,"P001, P002",13.0,3827.00
5,Retail,Maharashtra,1,MAHINDRA & MAHINDRA,Paid Service,7,1.0,2.0,super_slow,"P001, P002",50.0,3827.00
9,Retail,Maharashtra,0,VOLKSWAGEN,Running Repairs,1,1.0,2.0,mid,P001,85.0,35132.65
10,Retail,Maharashtra,0,VOLKSWAGEN,Running Repairs,1,1.0,2.0,fast,P001,300.6,35132.65


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60552 entries, 0 to 225636
Data columns (total 12 columns):
Cust Type               60552 non-null object
State                   60552 non-null object
KMs Reading             60552 non-null int32
Make                    60552 non-null object
Order Type              60552 non-null object
Invoice month           60552 non-null int64
Partner Type            60552 non-null float64
Title                   60552 non-null float64
ServiceTime_category    60552 non-null object
Item Category           60552 non-null object
Order Quantity          60552 non-null float64
LTV                     60552 non-null float64
dtypes: float64(4), int32(1), int64(1), object(6)
memory usage: 5.8+ MB


In [101]:
cat_df = df.select_dtypes(include = "object")

In [102]:
cat_df.head()

,Cust Type,State,Make,Order Type,ServiceTime_category,Item Category
0,Retail,Maharashtra,FORD,Paid Service,super_fast,"P001, P002"
4,Retail,Maharashtra,MAHINDRA & MAHINDRA,Running Repairs,super_fast,"P001, P002"
5,Retail,Maharashtra,MAHINDRA & MAHINDRA,Paid Service,super_slow,"P001, P002"
9,Retail,Maharashtra,VOLKSWAGEN,Running Repairs,mid,P001
10,Retail,Maharashtra,VOLKSWAGEN,Running Repairs,fast,P001


In [103]:
dummy = pd.get_dummies(cat_df)
final_df = pd.concat([df,dummy],1)

In [104]:
final_df.drop(list(cat_df),1, inplace = True)

In [105]:
final_df.head()

,KMs Reading,Invoice month,Partner Type,Title,Order Quantity,LTV,Cust Type_Corporate others,Cust Type_Corporate- M&M,Cust Type_Retail,State_Andaman and Nico.In.,State_Andhra Pradesh,State_Arunachal Pradesh,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jammu and Kashmir,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,Make_AUDI,Make_BMW,Make_DAEWOO,Make_FIAT,Make_FORCE,Make_FORD,Make_GENERAL MOTORS,Make_HINDUSTAN MOTORS,Make_HONDA,Make_HYUNDAI,Make_JEEP,Make_LAND ROVER,Make_MAHINDRA & MAHINDRA,Make_MARUTI SUZUKI,Make_MERCEDES BENZ,Make_MITSUBISHI MOTORS,Make_NISSAN,Make_PORCHE,Make_PREMIER AUTOMOBILES,Make_RENAULT,Make_SKODA,Make_SONALIKA,Make_TATA MOTORS,Make_TOYOTA,Make_VOLKSWAGEN,Make_VOLVO,Order Type_Accidental,Order Type_Paid Service,Order Type_Repeat Order,Order Type_Running Repairs,Order Type_SMC Redemption,Order Type_SMC Value Package,ServiceTime_category_fast,ServiceTime_category_mid,ServiceTime_category_slow,ServiceTime_category_super_fast,ServiceTime_category_super_slow,Item Category_G2TX,"Item Category_G2TX, P011",Item Category_P001,"Item Category_P001, G2TX","Item Category_P001, G2TX, P011","Item Category_P001, P002","Item Category_P001, P002, G2TX","Item Category_P001, P002, G2TX, P011","Item Category_P001, P002, P010","Item Category_P001, P002, P010, G2TX","Item Category_P001, P002, P010, P011","Item Category_P001, P002, P011","Item Category_P001, P010","Item Category_P001, P010, P011","Item Category_P001, P011",Item Category_P002,"Item Category_P002, G2TX","Item Category_P002, G2TX, P011","Item Category_P002, P010","Item Category_P002, P010, G2TX","Item Category_P002, P010, P011","Item Category_P002, P011",Item Category_P010,"Item Category_P010, G2TX, P011","Item Category_P010, G2TX, P011, P001, P002","Item Category_P010, P001, P002, G2TX","Item Category_P010, P011",Item Category_P011
0,0,6,1.0,2.0,45.5,2201.78,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,5,1.0,2.0,13.0,3827.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,7,1.0,2.0,50.0,3827.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,1,1.0,2.0,85.0,35132.65,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,1,1.0,2.0,300.6,35132.65,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
scalar = MinMaxScaler()
new_df = scalar.fit_transform(final_df)
scl_df = pd.DataFrame(new_df,columns=final_df.columns)

In [107]:
scl_df.head()

,KMs Reading,Invoice month,Partner Type,Title,Order Quantity,LTV,Cust Type_Corporate others,Cust Type_Corporate- M&M,Cust Type_Retail,State_Andaman and Nico.In.,State_Andhra Pradesh,State_Arunachal Pradesh,State_Bihar,State_Chandigarh,State_Chhattisgarh,State_Daman and Diu,State_Delhi,State_Goa,State_Gujarat,State_Haryana,State_Himachal Pradesh,State_Jammu and Kashmir,State_Jharkhand,State_Karnataka,State_Kerala,State_Lakshadweep,State_Madhya Pradesh,State_Maharashtra,State_Puducherry,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Tripura,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal,Make_AUDI,Make_BMW,Make_DAEWOO,Make_FIAT,Make_FORCE,Make_FORD,Make_GENERAL MOTORS,Make_HINDUSTAN MOTORS,Make_HONDA,Make_HYUNDAI,Make_JEEP,Make_LAND ROVER,Make_MAHINDRA & MAHINDRA,Make_MARUTI SUZUKI,Make_MERCEDES BENZ,Make_MITSUBISHI MOTORS,Make_NISSAN,Make_PORCHE,Make_PREMIER AUTOMOBILES,Make_RENAULT,Make_SKODA,Make_SONALIKA,Make_TATA MOTORS,Make_TOYOTA,Make_VOLKSWAGEN,Make_VOLVO,Order Type_Accidental,Order Type_Paid Service,Order Type_Repeat Order,Order Type_Running Repairs,Order Type_SMC Redemption,Order Type_SMC Value Package,ServiceTime_category_fast,ServiceTime_category_mid,ServiceTime_category_slow,ServiceTime_category_super_fast,ServiceTime_category_super_slow,Item Category_G2TX,"Item Category_G2TX, P011",Item Category_P001,"Item Category_P001, G2TX","Item Category_P001, G2TX, P011","Item Category_P001, P002","Item Category_P001, P002, G2TX","Item Category_P001, P002, G2TX, P011","Item Category_P001, P002, P010","Item Category_P001, P002, P010, G2TX","Item Category_P001, P002, P010, P011","Item Category_P001, P002, P011","Item Category_P001, P010","Item Category_P001, P010, P011","Item Category_P001, P011",Item Category_P002,"Item Category_P002, G2TX","Item Category_P002, G2TX, P011","Item Category_P002, P010","Item Category_P002, P010, G2TX","Item Category_P002, P010, P011","Item Category_P002, P011",Item Category_P010,"Item Category_P010, G2TX, P011","Item Category_P010, G2TX, P011, P001, P002","Item Category_P010, P001, P002, G2TX","Item Category_P010, P011",Item Category_P011
0,0.000000,0.454545,0.0,1.0,0.000529,0.002331,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.142857,0.363636,0.0,1.0,0.000151,0.004052,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.142857,0.545455,0.0,1.0,0.000581,0.004052,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,1.0,0.000987,0.037209,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,1.0,0.003492,0.037209,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [209]:
scl_df.shape

(60552, 102)

In [108]:
X = scl_df.drop(["LTV"],1)
y = scl_df["LTV"]

In [109]:
X_train,X_test,y_train,y_test = tts(X,y, random_state=42, test_size = 0.2)

In [110]:
knn = KNeighborsRegressor()
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)
r2_score_knn = r2_score(y_pred_knn,y_test)
r2_score_knn

0.31546981027693055

In [111]:
dt = DecisionTreeRegressor(max_depth=50)
dt.fit(X_train,y_train)
y_pred_dt = dt.predict(X_test)
r2_score_dt = r2_score(y_pred_dt,y_test)
r2_score_dt

0.4915166860898178

In [112]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred_lr = lr.predict(X_test)
r2_score_lr = r2_score(y_pred_lr,y_test)
r2_score_lr

-1.1366321632566567e-05

In [113]:
rd = Ridge()
rd.fit(X_train,y_train)
y_pred_rd = rd.predict(X_test)
r2_score_rd = r2_score(y_pred_rd,y_test)
r2_score_rd

0.2730394864451825

In [114]:
ls = Lasso()
ls.fit(X_train,y_train)
y_pred_ls = ls.predict(X_test)
r2_score_ls = r2_score(y_pred_ls,y_test)
r2_score_ls

-1.5565069918240338e+32

In [117]:
mae_knn = mean_absolute_error(y_test,y_pred_knn)
mae_dt = mean_absolute_error(y_test,y_pred_dt)
mae_lr = mean_absolute_error(y_test,y_pred_lr)
mae_rd = mean_absolute_error(y_test,y_pred_rd)
mae_ls = mean_absolute_error(y_test,y_pred_ls)

In [118]:
rmse_knn = np.sqrt(mean_squared_error(y_test,y_pred_knn))
rmse_dt = np.sqrt(mean_squared_error(y_test,y_pred_dt))
rmse_lr = np.sqrt(mean_squared_error(y_test,y_pred_lr))
rmse_rd = np.sqrt(mean_squared_error(y_test,y_pred_rd))
rmse_ls = np.sqrt(mean_squared_error(y_test,y_pred_ls))

In [119]:
acc_df = pd.DataFrame({
    'Model': ['KNN',"Decision Tree", "Linear Reg","Ridge","Lasso"],
    'MAE': [mae_knn, mae_dt, mae_lr, mae_rd, mae_ls],
    'RMSE': [rmse_knn, rmse_dt, rmse_lr, rmse_rd, rmse_ls]})

acc_df = acc_df.set_index('Model')
acc_df

,MAE,RMSE
Model,,
KNN,1.123300e-02,2.747795e-02
Decision Tree,1.243184e-02,2.996242e-02
Linear Reg,2.257899e+06,1.820810e+08
Ridge,1.161844e-02,2.920019e-02
Lasso,1.476825e-02,4.328484e-02
